In [1]:
# iPython Setting
%load_ext autoreload
%autoreload 2

# Imports
import pandas as pd
from cellstructure import Datasheet
from files import get_list_of_files_with_ext, join

# String Cleanup

In this section, the individual keywords extracted from the table are cleaned up

In [2]:
# Files and paths
path_to_excel_folder = "/Volumes/T7/thesis-data/test/selected_excel/"

file_list = get_list_of_files_with_ext(
    path_to_folder=path_to_excel_folder,
    ext="xlsx",
    verbose=False,
    randomise=True
)

single_file = file_list[0]

print(single_file)

/Volumes/T7/thesis-data/test/selected_excel/59bf894a49f3c.xlsx


In [3]:
ds = Datasheet()
ds.load_tables_from_excel(path_to_excel=single_file)

print(ds.name)

for table in ds.tables:
    print()
    print(table.name)
    print()
    print(table.raw_df)
    print()
    print(table.low)

59bf894a49f3c.xlsx

Table_1

           0      1      2      3       4       5      6
0     Eff(%)  Pm(W)  Vm(V)  Im(A)  Voc(V)  Isc(A)  FF(%)
1  18.5-18.6   2.87  0.528  5.428   0.632   5.751  78.94
2  18.4-18.5   2.86  0.528  5.408   0.632   5.731  78.84
3  18.3-18.4   2.84  0.528  5.378   0.631   5.712  78.77
4  18.2-18.3   2.83  0.527  5.363   0.631   5.694  78.67
5  18.1-18.2   2.81  0.526  5.338   0.630   5.678  78.54
6  18.0-18.1   2.79  0.525  5.322   0.629   5.664  78.39
7  17.8-18.0   2.78  0.523  5.301   0.629   5.640  78.19
8  17.6-17.8   2.74  0.521  5.266   0.628   5.610  77.90

['Eff', 'Pm', 'W', 'Vm', 'V', 'Im', 'A', 'Voc', 'V', 'Isc', 'A', 'FF']

Table_2

           0                                              1          2  \
0  CELL TYPE                                          a(mm)      b(mm)   
1       M652                                       0.5\n125  62    0.5   
2   Comments                    a——Side length of the cell;              
3             b——Center

# Keyword Modelling

The the keywords and their labels are converted to sentences. It is not the most efficient way to do this, but it will get the job done. Needs to be reformulated if there are a large number of files and tables.

In [4]:
labels_file = "/Volumes/T7/thesis-data/test/selected_excel/labels.csv"

labels_df = pd.read_csv(labels_file, names=['filename', 'tablename', 'class'])
labels_list = labels_df.values.tolist()

ds = Datasheet()

useful_list = []

for label in labels_list:

    path_to_ds = join(path_to_excel_folder, label[0])

    ds.load_tables_from_excel(path_to_ds)

    for table in ds.tables:

        if table.name == label[1]:

            sow = ' '.join(table.low)

            useful_list.append([sow, label[2]])


useful_df = pd.DataFrame(useful_list, columns=['keywords', 'class'])
useful_df.head()

,keywords,class
0,Efficiency Code Unit Efficiency Eff Power Ppm ...,e
1,Dimension mm mm mm Thickness Si m m Front Sili...,d
2,Efficiency Code Unit Efficiency Eff Power Ppm ...,e
3,Light Intensity Dependence Intensity Wm Vpm Ipm,o
4,Temperature Coefficients Current Temperature C...,t


In [5]:
path_to_useful_csv = "/Volumes/T7/thesis-data/test/selected_excel/cleaned-up-2.csv"
useful_df.to_csv(path_to_useful_csv, header=False, index=False)

# Fitting the Model

In this section we will try our hand at this prediction thing

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB

In [7]:
path_to_useful_csv = "/Volumes/T7/thesis-data/test/selected_excel/cleaned-up-2.csv"

useful_df = pd.read_csv(path_to_useful_csv, names=['keywords', 'class'])
useful_df.fillna(value="", inplace=True)
useful_df.head()

,keywords,class
0,Efficiency Code Unit Efficiency Eff Power Ppm ...,e
1,Dimension mm mm mm Thickness Si m m Front Sili...,d
2,Efficiency Code Unit Efficiency Eff Power Ppm ...,e
3,Light Intensity Dependence Intensity Wm Vpm Ipm,o
4,Temperature Coefficients Current Temperature C...,t


In [8]:
cv = CountVectorizer(max_features=5000)
tf = TfidfVectorizer(max_features=5000)

X = useful_df['keywords']
y = useful_df['class']

X = tf.fit_transform(X)

X

<215x521 sparse matrix of type '<class 'numpy.float64'>'
	with 2748 stored elements in Compressed Sparse Row format>

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=None)

In [10]:
knn = KNeighborsClassifier(n_neighbors=7)

nb = MultinomialNB()

knn.fit(X_train, y_train)
nb.fit(X_train, y_train)

MultinomialNB()

In [11]:
knn_pred = knn.predict(X_test)
nb_pred = nb.predict(X_test)

In [12]:
print("K Nearest Neighbours")
print(classification_report(y_test, knn_pred))

print("Naive Bayes")
print(classification_report(y_test, nb_pred))

K Nearest Neighbours
              precision    recall  f1-score   support

           d       1.00      0.67      0.80         6
           e       1.00      0.70      0.82        10
           o       0.75      1.00      0.86        18
           t       1.00      0.89      0.94         9

    accuracy                           0.86        43
   macro avg       0.94      0.81      0.86        43
weighted avg       0.90      0.86      0.86        43

Naive Bayes
              precision    recall  f1-score   support

           d       1.00      1.00      1.00         6
           e       0.91      1.00      0.95        10
           o       0.94      0.94      0.94        18
           t       1.00      0.89      0.94         9

    accuracy                           0.95        43
   macro avg       0.96      0.96      0.96        43
weighted avg       0.96      0.95      0.95        43



In [13]:
import pickle

In [18]:
# Save the classification model
pickle.dump(nb, open("nb_classifier.pickle", "wb"))

# Save the Vectoriser
pickle.dump(tf, open("vectoriser.pickle", "wb"))

In [19]:
clf = pickle.load(open("nb_classifier.pickle", "rb"))
vec = pickle.load(open("vectoriser.pickle", "rb"))

print(clf)
print(vec)

MultinomialNB()
TfidfVectorizer(max_features=5000)


In [20]:
clf_pred = clf.predict(X_test)
print(clf_pred)
print("Naive Bayes")
print(classification_report(y_test, nb_pred))

['o' 'o' 'e' 'o' 'o' 'd' 'e' 't' 'o' 'o' 'e' 't' 'd' 't' 'd' 'o' 'e' 'o'
 'd' 'o' 'e' 't' 'o' 't' 'o' 'o' 't' 'e' 't' 'o' 'o' 'e' 'e' 'o' 'd' 'e'
 't' 'e' 'o' 'o' 'd' 'e' 'o']
Naive Bayes
              precision    recall  f1-score   support

           d       1.00      1.00      1.00         6
           e       0.91      1.00      0.95        10
           o       0.94      0.94      0.94        18
           t       1.00      0.89      0.94         9

    accuracy                           0.95        43
   macro avg       0.96      0.96      0.96        43
weighted avg       0.96      0.95      0.95        43



In [33]:
sample_ds = Datasheet()
sample_ds.load_tables_from_excel("/Volumes/T7/thesis-data/test/single_excel/test.xlsx")

low = sample_ds.tables[0].low

sow = ' '.join(low)

print(sow)

print(low)

tow = vec.transform([sow])

print(clf.predict(tow))

Electrical Performance No Efficiency Pmpp W Umpp V Impp A Uoc v Isc A
['Electrical', 'Performance', 'No', 'Efficiency', 'Pmpp', 'W', 'Umpp', 'V', 'Impp', 'A', 'Uoc', 'v', 'Isc', 'A']
['e']


In [43]:
sample_ds_2 = Datasheet(
    path_to_excel="/Volumes/T7/thesis-data/test/single_excel/Allesun_AV-166-9M.xlsx",
    path_to_clf="nb_classifier.pickle",
    path_to_vec="vectoriser.pickle"
    )

d
o
t
e
